# Python для анализа данных

*Алла Тамбовцева, НИУ ВШЭ*

дополнения: *Ян Пиле, НИУ ВШЭ*

### Работа с `selenium`: продолжение

Сегодня мы продолжим работать над задачей, поставленной на прошлом занятии ‒ выгрузка адресов всех участковых избирательных комиссий Ивановской области. Сначала загрузим все необходимые для работы библиотеки:

* `selenium` ‒ для автоматизации работы в браузере
* `re` ‒ для поиска адреса на странице с помощью регулярных выражений 
* `time` ‒ для добавления задержки
* `pandas` ‒ для сохранения результатов в датафрейм

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys

In [ ]:
from webdriver_manager.chrome import ChromeDriverManager

driver = webdriver.Chrome(ChromeDriverManager().install())

In [ ]:
import re
from time import sleep
import pandas as pd
from bs4 import BeautifulSoup

In [ ]:
driver.get("http://cikrf.ru/digital-services/naydi-svoy-izbiratelnyy-uchastok/")

fld = driver.find_element_by_xpath("/html/body/div[2]/div/div/button")
fld.click()

Теперь напишем функцию `get_uik_address()`, которая принимает на вход два аргумента, номер участка и регион, и возвращает строку с адресом. Для этого в тело функции скопируем код с прошлого занятия:

In [ ]:
def get_uik_address(n_uik, reg):
    
    driver.get("http://cikrf.ru/digital-services/naydi-svoy-izbiratelnyy-uchastok/")

#     fld = driver.find_element_by_xpath("/html/body/div[2]/div/div/button")
#     fld.click()

#     sleep(1.5)

    fld = driver.find_element_by_xpath("/html/body/div[1]/div/div[2]/div/div[4]/form/div[1]/div[2]")
    fld.click()

    sleep(1.5)

    fld = driver.find_element_by_xpath('/html/body/div[1]/div/div[2]/div/div[4]/form/div[3]/div[1]/div/span/span[1]/span/span[2]')
    fld.click()

    sleep(1.5)

    reg_field = driver.find_element_by_xpath('/html/body/span/span/span[1]/input')
    reg_field.send_keys(reg)

    sleep(1.5)

    reg_field.send_keys(Keys.RETURN)

    sleep(1.5)

    uik_field = driver.find_element_by_xpath('/html/body/div[1]/div/div[2]/div/div[4]/form/div[3]/div[2]/div/input')
    uik_field.click()
    uik_field.clear()
    uik_field.send_keys(n_uik)

    sleep(1.5)

    uik_field.send_keys(Keys.RETURN)
    
    sleep(1.5)
    
    my_page = BeautifulSoup(driver.page_source)
    p = filter(lambda x: x not in ('',None),[i.text for i in my_page.find_all('span',{'class':'address'})])
    p = next(p)
    
    if p is None or p=='':
        p = 'Адреса не нашлось'
        
    return p

Теперь попробуем взять несколько номеров участков и посмотреть, что получается в цикле. Только давайте перестрахуемся ‒ напишем выражение с исключением, чтобы в случае, если страница не содержит адреса или загружается некорректно, наш код не ломался и не происходило выхода из цикла. В случае, если все хорошо (адрес есть), Python будет его сохранять («ветка» c `try`), в случае, если все плохо (адреса нет ни в каком виде), Python будет записывать вместо него пустую строку (ветка с `except`) и двигаться дальше. 

In [ ]:
uiks = range(240, 245)

In [ ]:
get_uik_address(244, "Ивановская область")

In [ ]:
addresses = []

for u in uiks:
    try:
        address = get_uik_address(u, "Ивановская область")
        print(address)
    except:
        address = ""
    addresses.append(address)
    print(u, address)

Работает! Создадим список со всеми номерами избирательных участков Ивановской области:

In [ ]:
addresses

**Важно:** периодически открывайте окно браузера, в котором Python ищет избирательные участки! Это не только приятно (смотреть, как в полях для поиска все заполняется без нашего участия), но и полезно: так можно заметить, если что-то пошло не так. 

Создадим датафрейм из словаря, ключами которого служат названия столбцов таблицы, а значениями – списки элементов этих столбцов.

In [ ]:
df = pd.DataFrame({'uik': uiks, 'address': addresses})

In [ ]:
df.head()

In [ ]:
list(df.address)

Сохраним таблицу в csv-файл:

In [ ]:
df.to_csv('Ivanovo.csv', index=False)

In [ ]:
driver.close()